In [3]:
import os
import random
import tensorflow as tf
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
import keras

Let's see how many photos we have to train, test and validate our model.

In [4]:
train_folder= r"C:\Users\meiza\Documents\GitHub\CNN-Pneumonia\chest_xray\train"
val_folder = r"C:\Users\meiza\Documents\GitHub\CNN-Pneumonia\chest_xray\val"
test_folder = r"C:\Users\meiza\Documents\GitHub\CNN-Pneumonia\chest_xray\test"

train_normal_folder= r"C:\Users\meiza\Documents\GitHub\CNN-Pneumonia\chest_xray\train\NORMAL"
val_normal_folder = r"C:\Users\meiza\Documents\GitHub\CNN-Pneumonia\chest_xray\val\NORMAL"
test_normal_folder = r"C:\Users\meiza\Documents\GitHub\CNN-Pneumonia\chest_xray\test\NORMAL"

train_pneumonia_folder= r"C:\Users\meiza\Documents\GitHub\CNN-Pneumonia\chest_xray\train\PNEUMONIA"
val_pneumonia_folder = r"C:\Users\meiza\Documents\GitHub\CNN-Pneumonia\chest_xray\val\PNEUMONIA"
test_pneumonia_folder = r"C:\Users\meiza\Documents\GitHub\CNN-Pneumonia\chest_xray\test\PNEUMONIA"

print(len(os.listdir(train_normal_folder)))
print(len(os.listdir(val_normal_folder)))
print(len(os.listdir(test_normal_folder)))
print(len(os.listdir(train_pneumonia_folder)))
print(len(os.listdir(val_pneumonia_folder)))
print(len(os.listdir(test_pneumonia_folder)))

1341
8
234
3875
8
390


We create the model composed of convolution, max pooling, flatten, and dense layers.

In [5]:
model = keras.models.Sequential([
    keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Conv2D(32, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


Next step consists in generating and transforming our dataset to fit it to our model.

In [6]:
train_datagen = ImageDataGenerator(
                                    rescale=1/255,featurewise_center=False,
                                    samplewise_center=False,
                                    featurewise_std_normalization=False,
                                    samplewise_std_normalization=False,
                                    zca_whitening=False,
                                    rotation_range = 30,
                                    zoom_range = 0.2,
                                    width_shift_range=0.1,
                                    height_shift_range=0.1,
                                    horizontal_flip = False,
                                    vertical_flip=False)

val_datagen = ImageDataGenerator(rescale=1/255)

test_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
                                                    train_folder,
                                                    target_size=(150, 150),
                                                    batch_size=32,
                                                    class_mode='categorical')

val_generator = val_datagen.flow_from_directory(
                                                    val_folder,
                                                    target_size=(150, 150),
                                                    batch_size=32,
                                                    class_mode='categorical')
test_generator = test_datagen.flow_from_directory(
                                                    test_folder,
                                                    target_size=(150, 150),
                                                    batch_size=32,
                                                    class_mode='categorical')



Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


Let's train our model.

In [37]:
history = model.fit(train_generator,
                              epochs=10,
                              verbose=1,
                              validation_data=val_generator)

Epoch 1/4
163/163 [==============================] - 129s 790ms/step - loss: 0.4067 - accuracy: 0.8181 - val_loss: 0.5215 - val_accuracy: 0.8125
Epoch 2/4
163/163 [==============================] - 115s 703ms/step - loss: 0.2386 - accuracy: 0.9005 - val_loss: 0.6898 - val_accuracy: 0.6875
Epoch 3/4
163/163 [==============================] - 120s 738ms/step - loss: 0.2230 - accuracy: 0.9101 - val_loss: 0.4376 - val_accuracy: 0.9375
Epoch 4/4
163/163 [==============================] - 135s 831ms/step - loss: 0.1841 - accuracy: 0.9287 - val_loss: 0.6788 - val_accuracy: 0.6875


Let's take a look at the following graphs. We can see that the more epochs we run the more accuracy we achieve.

In [7]:
%matplotlib inline

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()



plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")
plt.figure()

NameError: name 'history' is not defined

Now it's time to calculate the accuracy with test dataset.

In [39]:
print("Loss of the model is - " , model.evaluate(test_generator)[0]*100 , "%")
print("Accuracy of the model is - " , model.evaluate(test_generator)[1]*100 , "%")

20/20 [==============================] - 8s 389ms/step - loss: 0.2456 - accuracy: 0.9167
Loss of the model is -  24.557198584079742 %
20/20 [==============================] - 7s 349ms/step - loss: 0.2456 - accuracy: 0.9167
Accuracy of the model is -  91.66666865348816 %


Finally and once we get our expected model, we save it to avoid having to train again.

In [40]:
model.save('model.h5')